In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker
import time
import sys

# Initialize faker for realistic data
fake = Faker()

# Simple progress bar implementation
class ProgressBar:
    def __init__(self, total, desc=""):
        self.total = total
        self.current = 0
        self.desc = desc
        self.start_time = time.time()
        
    def update(self, n=1):
        self.current += n
        self.display()
        
    def display(self):
        percent = self.current / self.total * 100
        bar_length = 50
        filled_length = int(bar_length * self.current // self.total)
        bar = '█' * filled_length + '░' * (bar_length - filled_length)
        
        elapsed_time = time.time() - self.start_time
        if self.current > 0:
            items_per_sec = self.current / elapsed_time
            eta = (self.total - self.current) / items_per_sec if items_per_sec > 0 else 0
            eta_str = f"ETA: {self.format_time(eta)}"
        else:
            eta_str = "ETA: --:--:--"
            
        sys.stdout.write(f'\r{self.desc} |{bar}| {percent:.1f}% ({self.current}/{self.total}) | Speed: {items_per_sec:.1f} rec/s | {eta_str}')
        sys.stdout.flush()
        
    def format_time(self, seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours:02d}:{minutes:02d}:{secs:02d}"
        
    def close(self):
        elapsed = time.time() - self.start_time
        sys.stdout.write(f'\r{self.desc} |{"█" * 50}| 100.0% ({self.total}/{self.total}) | Time: {self.format_time(elapsed)}        \n')
        sys.stdout.flush()

# Generate column names based on your provided headers
def get_delivered_columns():
    headers = [
        "ReceiptID", "WarehouseID", "OrderID", "TenderID", "SupplierID", 
        "ReceiptDate", "Level1Category", "Level2Category", "Level3Category", 
        "ItemCode", "Remarks", "UnitOfMeasure", "ShapeCode", "SourceCode", 
        "BatchNo", "PackSize", "OnHold", "IsStop", "ExpiryDate", "PaletteID", 
        "Locator", "QtyOnHand", "QtyReserved", "QtyAvailable", "UnitCost", 
        "QtyOnHold", "SalesUnit", "TotalSales", "PackageType", "QtyPerPalette",
        "Length_cm", "Width_cm", "Height_cm", "Volume_cm3", "TotalPallets",
        "Weight_kg", "SupplierName", "SupplierAddress", "Manufacturer",
        "SupplierContact", "SuppliedDate", "WarehouseLocation", 
        "ProcurementFee", "LPO_Number"  # Unique identifier
    ]
    return headers

def get_issued_columns():
    headers = [
        "IssueID", "LPO_Number", "ItemCode", "IssueDate", "IssuedTo", 
        "IssuedToID", "IssuedBy", "IssuedQuantity", "UnitCost", "TotalCost", 
        "IssueType", "TransportMode", "DeliveryStatus", "WaybillNo", "Remarks", 
        "Department", "RequestID", "BatchNo", "ExpiryDate", "StorageCondition", 
        "QualityCheck", "IssuedFromWarehouse"
    ]
    return headers

# Generate warehouse data for medical commodities
def generate_delivered_data(num_records=5000):
    """Generate dataset for medical commodities delivered"""
    
    print(f"\n{'='*60}")
    print("GENERATING DELIVERY DATA")
    print(f"{'='*60}")
    
    # Medical categories for realistic data
    medical_categories = {
        "Level1": ["Pharmaceuticals", "Medical Equipment", "Consumables", "Laboratory"],
        "Level2": {
            "Pharmaceuticals": ["Antibiotics", "Analgesics", "Antivirals", "Vaccines"],
            "Medical Equipment": ["Diagnostic", "Therapeutic", "Monitoring"],
            "Consumables": ["Syringes", "Gloves", "Bandages", "Masks"],
            "Laboratory": ["Reagents", "Test Kits", "Specimen Containers"]
        },
        "Level3": {
            "Antibiotics": ["Amoxicillin", "Ciprofloxacin", "Azithromycin"],
            "Analgesics": ["Paracetamol", "Ibuprofen", "Aspirin"],
            "Antivirals": ["Acyclovir", "Oseltamivir"],
            "Vaccines": ["COVID-19", "Influenza", "Hepatitis B"],
            "Diagnostic": ["Stethoscope", "Thermometer", "BP Monitor"],
            "Therapeutic": ["Nebulizer", "Oxygen Concentrator"],
            "Monitoring": ["Pulse Oximeter", "Glucometer"],
            "Syringes": ["1ml", "5ml", "10ml"],
            "Gloves": ["Nitrile", "Latex", "Vinyl"],
            "Bandages": ["Crepe", "Elastic", "Gauze"],
            "Masks": ["N95", "Surgical", "Cloth"],
            "Reagents": ["Biochemical", "Hematology", "Immunology"],
            "Test Kits": ["Malaria", "HIV", "COVID-19"],
            "Specimen Containers": ["Urine", "Blood", "Stool"]
        }
    }
    
    print(f"Generating {num_records:,} delivery records...")
    progress_bar = ProgressBar(num_records, "Creating delivery records")
    
    data = []
    columns = get_delivered_columns()
    
    # Generate unique LPO numbers for delivery
    lpo_numbers = [f"LPO-{str(i).zfill(6)}" for i in range(1, num_records + 1)]
    
    for i in range(num_records):
        # Randomly select category hierarchy
        level1 = random.choice(medical_categories["Level1"])
        level2 = random.choice(medical_categories["Level2"][level1])
        level3_options = medical_categories["Level3"].get(level2, ["General"])
        level3 = random.choice(level3_options) if level3_options else "General"
        
        # Generate realistic dates
        receipt_date = fake.date_between(start_date='-2y', end_date='today')
        expiry_date = receipt_date + timedelta(days=random.randint(180, 1080))
        
        # Generate quantities
        qty = random.randint(10, 1000)
        qty_reserved = random.randint(0, int(qty * 0.3))
        qty_on_hold = random.randint(0, int(qty * 0.2))
        unit_cost = round(random.uniform(1.5, 500.0), 2)
        
        record = {
            "ReceiptID": f"RCPT-{str(i+1).zfill(6)}",
            "WarehouseID": f"WH-{random.choice(['A', 'B', 'C'])}",
            "OrderID": f"ORD-{str(i+1).zfill(6)}",
            "TenderID": f"TDR-{random.randint(1000, 9999)}",
            "SupplierID": f"SUP-{random.randint(100, 999)}",
            "ReceiptDate": receipt_date,
            "Level1Category": level1,
            "Level2Category": level2,
            "Level3Category": level3,
            "ItemCode": f"MED-{level3[:3].upper()}-{random.randint(1000, 9999)}",
            "Remarks": random.choice(["", "Priority", "Fragile", "Refrigerated", "Urgent"]),
            "UnitOfMeasure": random.choice(["Box", "Piece", "Pack", "Bottle", "Vial"]),
            "ShapeCode": random.choice(["RECT", "CYL", "SQR", "IRR"]),
            "SourceCode": random.choice(["LOCAL", "IMPORT", "DONATION"]),
            "BatchNo": f"BATCH-{random.randint(10000, 99999)}",
            "PackSize": random.randint(1, 100),
            "OnHold": random.choice([True, False]),
            "IsStop": random.choice([True, False]),
            "ExpiryDate": expiry_date,
            "PaletteID": f"PAL-{random.randint(100, 999)}",
            "Locator": f"LOC-{random.choice(['A', 'B', 'C'])}{random.randint(1, 50)}",
            "QtyOnHand": qty,
            "QtyReserved": qty_reserved,
            "QtyAvailable": qty - qty_reserved - qty_on_hold,
            "UnitCost": unit_cost,
            "QtyOnHold": qty_on_hold,
            "SalesUnit": random.choice(["Box", "Piece", "Carton"]),
            "TotalSales": round(qty * unit_cost * random.uniform(0.8, 1.2), 2),
            "PackageType": random.choice(["Cardboard", "Plastic", "Wooden"]),
            "QtyPerPalette": random.randint(10, 100),
            "Length_cm": random.randint(10, 200),
            "Width_cm": random.randint(10, 100),
            "Height_cm": random.randint(5, 50),
            "Volume_cm3": 0,  # Will calculate
            "TotalPallets": random.randint(1, 20),
            "Weight_kg": round(random.uniform(0.1, 50.0), 2),
            "SupplierName": fake.company(),
            "SupplierAddress": fake.address().replace('\n', ', '),
            "Manufacturer": fake.company(),
            "SupplierContact": fake.phone_number(),
            "SuppliedDate": receipt_date - timedelta(days=random.randint(1, 7)),
            "WarehouseLocation": random.choice(["Main", "Cold Storage", "Quarantine", "Dispatch"]),
            "ProcurementFee": round(random.uniform(10.0, 500.0), 2),
            "LPO_Number": lpo_numbers[i]
        }
        
        # Calculate volume
        record["Volume_cm3"] = record["Length_cm"] * record["Width_cm"] * record["Height_cm"]
        data.append(record)
        
        progress_bar.update(1)
    
    progress_bar.close()
    
    df = pd.DataFrame(data, columns=columns)
    
    print(f"\n✓ Generated {len(df):,} delivery records")
    print(f"✓ Columns: {len(columns)}")
    print(f"✓ Date range: {df['ReceiptDate'].min()} to {df['ReceiptDate'].max()}")
    print(f"✓ Unique LPO numbers: {df['LPO_Number'].nunique()}")
    print(f"✓ Total inventory value: ${df['TotalSales'].sum():,.2f}")
    
    return df, lpo_numbers

def generate_issued_data(delivered_lpos, num_records=3500):
    """Generate issued commodities dataset with some common LPO numbers"""
    
    print(f"\n{'='*60}")
    print("GENERATING ISSUANCE DATA")
    print(f"{'='*60}")
    
    print(f"Generating {num_records:,} issuance records...")
    
    # Determine overlap between datasets
    overlap_percentage = 0.4  # 40% of issued records share LPOs with delivered
    overlap_count = int(num_records * overlap_percentage)
    
    # Get LPOs for overlap
    common_lpos = random.sample(delivered_lpos, min(overlap_count, len(delivered_lpos)))
    
    # Generate new LPOs for non-overlap
    start_idx = len(delivered_lpos) + 1
    new_lpos = [f"LPO-{str(i).zfill(6)}" for i in range(start_idx, start_idx + num_records - len(common_lpos))]
    
    # Combine all LPOs for issuance
    all_issued_lpos = common_lpos + new_lpos
    random.shuffle(all_issued_lpos)
    
    print(f"• {len(common_lpos):,} records share LPOs with delivery data")
    print(f"• {len(new_lpos):,} records have new LPOs")
    
    progress_bar = ProgressBar(num_records, "Creating issuance records")
    
    data = []
    columns = get_issued_columns()
    
    # Create a mapping of LPO to ItemCode for common records (simplified approach)
    lpo_item_map = {}
    for i in range(min(500, len(delivered_lpos))):
        lpo_item_map[delivered_lpos[i]] = f"MED-{random.choice(['ANT', 'ANL', 'VAC', 'EQU'])}-{random.randint(1000, 9999)}"
    
    for i in range(num_records):
        lpo = all_issued_lpos[i]
        
        # Determine if this is a common LPO
        if lpo in lpo_item_map:
            item_code = lpo_item_map[lpo]
            unit_cost = round(random.uniform(1.5, 500.0), 2)
        else:
            item_code = f"MED-ISS-{random.randint(1000, 9999)}"
            unit_cost = round(random.uniform(1.5, 500.0), 2)
        
        # Generate issued quantities
        issued_qty = random.randint(1, 100)
        issue_date = fake.date_between(start_date='-1y', end_date='today')
        
        record = {
            "IssueID": f"ISS-{str(i+1).zfill(6)}",
            "LPO_Number": lpo,
            "ItemCode": item_code,
            "IssueDate": issue_date,
            "IssuedTo": random.choice(["Hospital", "Clinic", "Pharmacy", "NGO", "Government Agency"]),
            "IssuedToID": f"REC-{random.randint(1000, 9999)}",
            "IssuedBy": fake.name(),
            "IssuedQuantity": issued_qty,
            "UnitCost": unit_cost,
            "TotalCost": round(issued_qty * unit_cost, 2),
            "IssueType": random.choice(["Regular", "Emergency", "Donation", "Return", "Loan"]),
            "TransportMode": random.choice(["Road", "Air", "Sea", "Courier"]),
            "DeliveryStatus": random.choice(["Delivered", "In Transit", "Pending", "Partially Delivered"]),
            "WaybillNo": f"WB-{random.randint(10000, 99999)}",
            "Remarks": random.choice(["", "Urgent", "Fragile", "Temperature Sensitive", "Controlled Substance"]),
            "Department": random.choice(["ER", "ICU", "OPD", "Surgery", "Laboratory", "Pediatrics", "Maternity"]),
            "RequestID": f"REQ-{random.randint(1000, 9999)}",
            "BatchNo": f"BATCH-{random.randint(10000, 99999)}",
            "ExpiryDate": issue_date + timedelta(days=random.randint(30, 365)),
            "StorageCondition": random.choice(["Room Temperature", "Cold Chain", "Refrigerated", "Frozen"]),
            "QualityCheck": random.choice(["Passed", "Failed", "Pending", "Conditional"]),
            "IssuedFromWarehouse": random.choice(["Main Warehouse", "Cold Storage", "Dispatch Center", "Satellite Store"])
        }
        data.append(record)
        
        progress_bar.update(1)
    
    progress_bar.close()
    
    df = pd.DataFrame(data, columns=columns)
    
    print(f"\n✓ Generated {len(df):,} issuance records")
    print(f"✓ Date range: {df['IssueDate'].min()} to {df['IssueDate'].max()}")
    print(f"✓ Total value issued: ${df['TotalCost'].sum():,.2f}")
    print(f"✓ Common LPOs with delivery data: {len(common_lpos):,}")
    
    return df

def generate_sample_sql_queries():
    """Generate SQL practice queries and save to file"""
    
    queries = """
-- =====================================================================
-- SQL PRACTICE QUERIES FOR WAREHOUSE MANAGEMENT DATASETS
-- =====================================================================

-- 1. BASIC QUERIES
-- =================
-- List all delivered items
SELECT * FROM commodities_delivered LIMIT 10;

-- List all issued items
SELECT * FROM commodities_issued LIMIT 10;

-- Count total items delivered
SELECT COUNT(*) as TotalDeliveries FROM commodities_delivered;

-- Count total items issued
SELECT COUNT(*) as TotalIssued FROM commodities_issued;

-- 2. SUBQUERIES PRACTICE
-- =======================
-- Find items delivered but not issued
SELECT d.* 
FROM commodities_delivered d
WHERE d.LPO_Number NOT IN (
    SELECT DISTINCT LPO_Number 
    FROM commodities_issued
)
ORDER BY d.ReceiptDate DESC;

-- Find items that have been issued more than once
SELECT LPO_Number, ItemCode, COUNT(*) as IssueCount
FROM commodities_issued
WHERE LPO_Number IN (
    SELECT LPO_Number 
    FROM commodities_delivered
    WHERE Level1Category = 'Pharmaceuticals'
)
GROUP BY LPO_Number, ItemCode
HAVING COUNT(*) > 1;

-- Find average unit cost by category using subquery
SELECT 
    Level1Category,
    AVG(UnitCost) as avg_cost,
    (SELECT AVG(UnitCost) FROM commodities_delivered) as overall_avg
FROM commodities_delivered
GROUP BY Level1Category;

-- 3. JOINS PRACTICE
-- ==================
-- INNER JOIN: Find complete delivery-issuance pairs
SELECT 
    d.LPO_Number,
    d.ItemCode,
    d.ReceiptDate,
    d.SupplierName,
    i.IssueDate,
    i.IssuedTo,
    i.IssuedQuantity,
    d.QtyOnHand
FROM commodities_delivered d
INNER JOIN commodities_issued i ON d.LPO_Number = i.LPO_Number;

-- LEFT JOIN: All deliveries with matching issuances
SELECT 
    d.LPO_Number,
    d.ItemCode,
    d.QtyOnHand,
    COALESCE(SUM(i.IssuedQuantity), 0) as total_issued,
    d.QtyOnHand - COALESCE(SUM(i.IssuedQuantity), 0) as remaining_stock
FROM commodities_delivered d
LEFT JOIN commodities_issued i ON d.LPO_Number = i.LPO_Number
GROUP BY d.LPO_Number, d.ItemCode, d.QtyOnHand
ORDER BY remaining_stock DESC;

-- 4. WINDOW FUNCTIONS PRACTICE
-- =============================
-- Rank suppliers by total value delivered
SELECT 
    SupplierName,
    COUNT(*) as delivery_count,
    SUM(TotalSales) as total_value,
    RANK() OVER (ORDER BY SUM(TotalSales) DESC) as supplier_rank,
    ROW_NUMBER() OVER (ORDER BY SUM(TotalSales) DESC) as row_num
FROM commodities_delivered
GROUP BY SupplierName;

-- Running total of issued quantities by date
SELECT 
    IssueDate,
    IssuedQuantity,
    TotalCost,
    SUM(IssuedQuantity) OVER (ORDER BY IssueDate) as running_quantity,
    SUM(TotalCost) OVER (ORDER BY IssueDate) as running_value,
    AVG(IssuedQuantity) OVER (ORDER BY IssueDate ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as weekly_avg_quantity
FROM commodities_issued
ORDER BY IssueDate;

-- 5. QUERY OPTIMIZATION PRACTICE
-- ===============================
-- Using EXISTS instead of IN
SELECT d.*
FROM commodities_delivered d
WHERE EXISTS (
    SELECT 1 
    FROM commodities_issued i 
    WHERE i.LPO_Number = d.LPO_Number
    AND i.IssueDate >= DATE('now', '-30 days')
);

-- Materialized query using CTE
WITH monthly_summary AS (
    SELECT 
        strftime('%Y-%m', ReceiptDate) as delivery_month,
        Level1Category,
        COUNT(*) as delivery_count,
        SUM(TotalSales) as monthly_value
    FROM commodities_delivered
    GROUP BY strftime('%Y-%m', ReceiptDate), Level1Category
)
SELECT 
    delivery_month,
    Level1Category,
    delivery_count,
    monthly_value,
    SUM(monthly_value) OVER (PARTITION BY Level1Category ORDER BY delivery_month) as ytd_value
FROM monthly_summary
ORDER BY delivery_month DESC, monthly_value DESC;

-- 6. RECURSIVE CTE PRACTICE (For databases that support it)
-- ============================================================
-- Note: SQLite supports recursive CTEs
-- Generate a date series for the last 30 days
WITH RECURSIVE date_series AS (
    SELECT DATE('now', '-30 days') as report_date
    UNION ALL
    SELECT DATE(report_date, '+1 day')
    FROM date_series
    WHERE report_date < DATE('now')
)
SELECT 
    ds.report_date,
    COUNT(DISTINCT d.LPO_Number) as deliveries_on_date,
    COUNT(DISTINCT i.IssueID) as issues_on_date
FROM date_series ds
LEFT JOIN commodities_delivered d ON ds.report_date = d.ReceiptDate
LEFT JOIN commodities_issued i ON ds.report_date = i.IssueDate
GROUP BY ds.report_date
ORDER BY ds.report_date;

-- 7. ADVANCED ANALYTICS
-- ======================
-- Stock turnover rate by category
SELECT 
    d.Level1Category,
    SUM(d.QtyOnHand) as total_stock,
    SUM(i.IssuedQuantity) as total_issued,
    CASE 
        WHEN SUM(d.QtyOnHand) > 0 
        THEN CAST(SUM(i.IssuedQuantity) AS FLOAT) / SUM(d.QtyOnHand) 
        ELSE 0 
    END as turnover_rate
FROM commodities_delivered d
LEFT JOIN commodities_issued i ON d.LPO_Number = i.LPO_Number
GROUP BY d.Level1Category
ORDER BY turnover_rate DESC;

-- Expiring soon analysis
SELECT 
    Level1Category,
    Level2Category,
    ItemCode,
    BatchNo,
    ExpiryDate,
    QtyOnHand,
    UnitCost,
    QtyOnHand * UnitCost as stock_value,
    julianday(ExpiryDate) - julianday('now') as days_to_expiry
FROM commodities_delivered
WHERE ExpiryDate BETWEEN DATE('now') AND DATE('now', '+90 days')
ORDER BY days_to_expiry, stock_value DESC;
"""
    
    return queries

def main():
    """Main function to generate warehouse datasets as CSV files"""
    
    print("=" * 70)
    print("WAREHOUSE MANAGEMENT DATASET GENERATOR")
    print("Generating CSV files for SQL Practice")
    print("=" * 70)
    
    # Configuration
    NUM_DELIVERY_RECORDS = 5000
    NUM_ISSUANCE_RECORDS = 3500
    
    print(f"\nConfiguration:")
    print(f"• Delivery records: {NUM_DELIVERY_RECORDS:,}")
    print(f"• Issuance records: {NUM_ISSUANCE_RECORDS:,}")
    print(f"• LPO overlap: ~40% (for join practice)")
    
    start_time = time.time()
    
    try:
        # Generate delivery data
        delivered_df, delivered_lpos = generate_delivered_data(NUM_DELIVERY_RECORDS)
        
        # Generate issued data
        issued_df = generate_issued_data(delivered_lpos, NUM_ISSUANCE_RECORDS)
        
        # Save to CSV files
        print(f"\n{'='*60}")
        print("SAVING CSV FILES")
        print(f"{'='*60}")
        
        delivered_file = 'commodities_delivered.csv'
        issued_file = 'commodities_issued.csv'
        
        # Save with different options for compatibility
        delivered_df.to_csv(delivered_file, index=False, encoding='utf-8')
        issued_df.to_csv(issued_file, index=False, encoding='utf-8')
        
        print(f"✓ {delivered_file}: {len(delivered_df):,} records, {delivered_df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
        print(f"✓ {issued_file}: {len(issued_df):,} records, {issued_df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
        
        # Generate SQL practice file
        sql_file = 'warehouse_sql_practice_queries.sql'
        sql_queries = generate_sample_sql_queries()
        
        with open(sql_file, 'w', encoding='utf-8') as f:
            f.write(sql_queries)
        
        print(f"✓ {sql_file}: Sample SQL queries for practice")
        
        elapsed_time = time.time() - start_time
        
        print(f"\n{'='*70}")
        print("GENERATION COMPLETE!")
        print(f"{'='*70}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        print(f"Total records generated: {len(delivered_df) + len(issued_df):,}")
        
        print(f"\nCSV FILES CREATED:")
        print(f"1. {delivered_file}")
        print(f"   • Columns: {len(delivered_df.columns)}")
        print(f"   • Key column: LPO_Number ({delivered_df['LPO_Number'].nunique():,} unique)")
        print(f"   • Date range: {delivered_df['ReceiptDate'].min()} to {delivered_df['ReceiptDate'].max()}")
        
        print(f"\n2. {issued_file}")
        print(f"   • Columns: {len(issued_df.columns)}")
        print(f"   • Key column: LPO_Number ({issued_df['LPO_Number'].nunique():,} unique)")
        print(f"   • Date range: {issued_df['IssueDate'].min()} to {issued_df['IssueDate'].max()}")
        
        print(f"\n3. {sql_file}")
        print(f"   • Contains SQL queries for practicing:")
        print(f"     - Subqueries")
        print(f"     - Joins (INNER, LEFT, etc.)")
        print(f"     - Window Functions")
        print(f"     - Query Optimization")
        print(f"     - Recursive CTEs")
        
        print(f"\nHOW TO USE:")
        print(f"1. Import CSV files into your preferred SQL database")
        print(f"2. Use LPO_Number as the common key for joins")
        print(f"3. Practice SQL concepts using the sample queries")
        
        print(f"\nRECOMMENDED PRACTICE EXERCISES:")
        print(f"• Find all items delivered but never issued")
        print(f"• Calculate total value of issued items by month")
        print(f"• Find suppliers with highest delivery volume")
        print(f"• Identify items nearing expiration")
        print(f"• Calculate stock turnover rate by category")
        
    except Exception as e:
        print(f"\n❌ Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

WAREHOUSE MANAGEMENT DATASET GENERATOR
Generating CSV files for SQL Practice

Configuration:
• Delivery records: 5,000
• Issuance records: 3,500
• LPO overlap: ~40% (for join practice)

GENERATING DELIVERY DATA
Generating 5,000 delivery records...
Creating delivery records |██████████████████████████████████████████████████| 100.0% (5000/5000) | Time: 00:00:06        TA: 00:00:00

✓ Generated 5,000 delivery records
✓ Columns: 44
✓ Date range: 2023-12-28 to 2025-12-27
✓ Unique LPO numbers: 5000
✓ Total inventory value: $638,897,433.44

GENERATING ISSUANCE DATA
Generating 3,500 issuance records...
• 1,400 records share LPOs with delivery data
• 2,100 records have new LPOs
Creating issuance records |█████████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 35.8% (1252/3500) | Speed: 1279.9 rec/s | ETA: 00:00:01

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Creating issuance records |██████████████████████████████████████████████████| 100.0% (3500/3500) | Time: 00:00:02        ETA: 00:00:00

✓ Generated 3,500 issuance records
✓ Date range: 2024-12-27 to 2025-12-27
✓ Total value issued: $43,800,234.79
✓ Common LPOs with delivery data: 1,400

SAVING CSV FILES
✓ commodities_delivered.csv: 5,000 records, 8.0 MB
✓ commodities_issued.csv: 3,500 records, 3.7 MB
✓ warehouse_sql_practice_queries.sql: Sample SQL queries for practice

GENERATION COMPLETE!
Total time: 8.81 seconds
Total records generated: 8,500

CSV FILES CREATED:
1. commodities_delivered.csv
   • Columns: 44
   • Key column: LPO_Number (5,000 unique)
   • Date range: 2023-12-28 to 2025-12-27

2. commodities_issued.csv
   • Columns: 22
   • Key column: LPO_Number (3,500 unique)
   • Date range: 2024-12-27 to 2025-12-27

3. warehouse_sql_practice_queries.sql
   • Contains SQL queries for practicing:
     - Subqueries
     - Joins (INNER, LEFT, etc.)
     - Window Functions
     - Quer